In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

import Metashape 

from easydict import EasyDict as edict

from lib.metashape import metashape_workflow_functions as meta
from lib.metashape import metashape_license_setup
from lib.metashape.read_yaml import read_yaml

# ---- Checl if Metashape is activated
# otherwise copy the license file from the full metashape install to the Metashape wheel module path, by running metashape_license_setup.
# Remember to change the path to the metashape.lic file in the metashape_license_setup script!
# metashape_license_setup
# Metashape.License().activate()
# Metashape.License().activate("HOST LMW-2E.POLIMI.IT any 2001")
print("Metashape is activated: ", Metashape.app.activated)


## Define where to get the config file (only used if running interactively)
cfg_path = "config/base.yml"
cfg = edict(read_yaml(cfg_path))

def write_markers_by_camera(chunk: Metashape.Chunk, 
                             file_name: str,
                             ) -> None:
    """ Write Marker image coordinates to csv file,
    sort by camera, as follows:
    cam1, marker1, x, y
    cam1, marker2, x, y
    ... 
    cam1, markerM, x, y
    cam2, marker1, x, y
    ....
    camN, markerM, x,Y
    
    Args:
        chunk (Metashape.Chunk): Metashape Chunk
        file_name (str): path of the output csv file
    """

    file = open(file_name, "wt")

    for camera in chunk.cameras:
        for marker in chunk.markers:
            projections = marker.projections  # list of marker projections
            marker_name = marker.label

            for cur_cam in marker.projections.keys():
                if cur_cam == camera:
                    cam_name = cur_cam.label
                    x, y = projections[cur_cam].coord
                    # writing output to file
                    file.write(cam_name + "," + marker_name + "," + "," +
                            "{:.4f}".format(x) + "," + "{:.4f}".format(y) + "\n")  
    file.close()
    print("Marker exported successfully")


def write_markers_by_marker(chunk: Metashape.Chunk,    file.close()

    sort by camera, as follows:
    marker1, cam1, x, y
    marker1, cam2, x, y
    ... 
    marker1, camN, x, y
    marker2, cam1, x, y
    ....
    markerM, camN, x, y
    
    Args:
        chunk (Metashape.Chunk): Metashape Chunk
        file_name (str): path of the output csv file
    """

    file = open(file_name, "wt")

    for marker in chunk.markers:
        projections = marker.projections  # list of marker projections
        marker_name = marker.label
        for camera in marker.projections.keys():
            x, y = projections[camera].coord
            label = camera.label
            # writing output to file
            file.write(marker_name + "," + label + "," +
                "{:.4f}".format(x) + "," + "{:.4f}".format(y) + "\n") 

    file.close()
    print("Marker exported successfully")


Metashape is activated:  True


In [ ]:
load_project = "data/belpy.psx"
project_file = "data/belpy_new.psx"
markers_file = "data/belpy_markers.csv"

# create a handle to the Metashape object
# When running via Metashape, can use: doc = Metashape.app.document
doc = Metashape.Document()

# If specified, open existing project
if load_project != "":
    doc.open(load_project)
else:
    # Initialize a chunk, set its CRS as specified
    chunk = doc.addChunk()
    chunk.crs = Metashape.CoordinateSystem(cfg["project_crs"])
    chunk.marker_crs = Metashape.CoordinateSystem(
        cfg["addGCPs"]["gcp_crs"])

# Save doc doc as new project (even if we opened an existing project, save as a separate one so the existing project remains accessible in its original state)
doc.save(project_file)

chunk = doc.chunk
write_markers_by_camera(chunk, markers_file)
